In [1]:
import pandas as pd
from scipy import stats
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [3]:
#df = pd.read_csv('data/corpus/220916-RecueilComplet.csv')

In [13]:
df = pd.read_csv("data/imports/recueil d'observation - articles labellisés.csv").iloc[1:]

In [14]:
df

,# site,# article,Site,url,Auteur/trice,Date de publication,Titre,explication,infox,signé,...,invérifiable,propos rapporté,sources citées,titre décalé,dénigrement,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné)
1,1.0,1.0,bonsens.org,https://bonsens.info/essais-pfizer-sur-le-vacc...,NaN,NaN,Essais Pfizer sur le vaccin anti-covid : le ra...,pseudo rapport par une pseudo scientifique mon...,oui,non,...,non,oui,oui,non,NaN,*,FB,vaccin,pseudo science,27 avril 22
2,2.0,2.0,breizh,https://www.breizh-info.com/2022/02/23/180457/...,NaN,23 février 22,Tests – vaccins – pass : fabrique de la paniqu...,yapa de pandémie et les vaccins ne marchent pas,oui,non,...,oui,non,non,oui,NaN,**,FB,vaccins,pseudo démonstration,mars 22
3,2.0,3.0,breizh,https://www.breizh-info.com/2022/02/20/180182/...,NaN,20 février 22,"Covid-19. La pandémie et les vaccins, pour mie...",yapa de pandémie et il y a un complot mondial,oui,oui,...,oui,oui,non,non,NaN,**,FB,vaccins,complot,mars 22
4,2.0,4.0,breizh,https://www.breizh-info.com/2022/04/19/185260/...,NaN,19 avril 2022,Covid-19. Une épidémie quasiment terminée avan...,démonstration que les vaccins n'ont servi à rien,oui,oui,...,oui,oui,non,non,NaN,**,FB,vaccins,complot,mars 22
5,2.0,5.0,breizh,https://www.breizh-info.com/2022/04/15/184984/...,NaN,15 avril 22,"Piscine, voile, ramadan : scènes de la vie ord...",absolument rien pour étayer cette histoire qui...,oui,non,...,oui,oui,non,non,NaN,*,FB,grand remplacement,histoire inventée,21 avril 22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,38.0,426.0,Wikistrike,https://www.wikistrike.com/2022/08/les-vaccine...,NaN,20 Aout 22,Les vaccinés infectés au Covid restent contagi...,L'article arrive a cette conclusion en utilisa...,oui,non,...,non,oui,oui,non,oui,**,CN,NaN,NaN,NaN
427,38.0,427.0,Wikistrike,https://www.wikistrike.com/2022/08/fresne-cett...,NaN,21 Aout 22,Fresnes: cette prison où les condamnés finisse...,NaN,non,non,...,non,oui,oui,non,non,NaN,CN,NaN,NaN,NaN
428,38.0,428.0,Wikistrike,https://www.wikistrike.com/2022/08/le-forum-ec...,NaN,23 Aout 22,Le Forum économique mondial estime qu’il exist...,Convaincre que le Forum economique mondial veu...,oui,non,...,non,oui,oui,non,oui,*,CN,NaN,NaN,NaN
429,38.0,429.0,Wikistrike,https://www.wikistrike.com/2022/08/le-masque-r...,NaN,24 Aout 22,"Le masque réduit les infections COVID de 0,06%",L'article prend une etude qui est loin d'etre ...,oui,non,...,non,non,non,non,non,**,CN,NaN,NaN,NaN


In [15]:
df.columns

Index(['# site', '# article', 'Site', 'url', 'Auteur/trice',
       'Date de publication', 'Titre', 'explication', 'infox', 'signé',
       'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
       'points de vue différents présentés', 'écriture de qualité',
       'intention', 'faits', 'opinions', 'fausse nouvelle', 'insinuations',
       'exagération', 'scientifique sulfureux', 'des pages et des pages',
       'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
       'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
       'dénigrement', 'degré infox', 'observateur', 'thème (abandonné)',
       'type infox (abandonné)', 'Date de collection (abandonné)'],
      dtype='object')

In [16]:
label_columns = ['signé',
       'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
       'points de vue différents présentés', 'écriture de qualité',
       'intention', 'faits', 'opinions', 'fausse nouvelle', 'insinuations',
       'exagération', 'scientifique sulfureux', 'des pages et des pages',
       'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
       'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',
        'degré infox']

Petite fonction pour normaliser quelques inconsistences dans les réponses (exemple : "Oui" et "oui").

In [17]:
def normalize(x):
    return x if type(x) != str else x.lower().strip()

In [18]:
y = df["infox"].apply(normalize)

In [19]:
pd.crosstab(df["lieux, dates et personnes cohérentes"].apply(normalize), df["infox"].apply(normalize), margins=True)

infox,non,oui,All
"lieux, dates et personnes cohérentes",,,
non,77,135,212
oui,131,87,218
All,208,222,430


La cellule ci-dessous effectue pour chaque colonne d'intérêt un test d'indépendence du χ². J'ai choisi le niveau α = 0.05 comme seuil, mais ça confirme ce que vous aviez vu : dans votre dataset la colonne `signé` est indépendante du caractère `infox` de l'article.

Par ailleurs, pour les variables binaires, j'ai calculé le "lift" ; j'aime bien car ça donne un peu une dimension prédictive. Ca permet de répondre à la question "si je n'utilisais que cette colonne pour prédire si mon article est une infox ou pas, quelle performance j'aurais ?".
C'est le pourcentage de précision qu'on gagnerait si on utilisait directement cette variable pour décider du caractère "intox" de l'article (mesuré en différence de points de pourcentage donc).
Cela prend en compte les "inversions de polarité", càd que si le "non" de la colonne est plus corrélé avec le fait d'être une intox, on utilise "non" plutôt que "oui".

Un exemple : si on utilise la colonne `lieux, dates et personnes cohérentes`, on voit ci-dessus que le "non" nous renseigne beaucoup sur le caractère infox, puisque dans ce cas, on a 63% de d'infox (134 / 210), à comparer avec le baseline de 52% (217 / 421) si on ne sait rien.
De manière symétrique, si cette colonne vaut "oui", on a 61% de chances que l'article ne soit pas une infox (128 / 211), par rapport à 48% (204/421).

In [20]:
alpha = 0.05

data = []

for col in label_columns:
    try:
        lift = np.fabs(accuracy_score(df[col].apply(normalize), df["infox"].apply(normalize)) - (y.value_counts() / len(y)).max())
    except:
        lift = None
    data.append((
        col,
        stats.chi2_contingency(pd.crosstab(df[col].apply(normalize), df["infox"].apply(normalize)))[1] < alpha,
        lift
    ))

pd.DataFrame(data, columns=["column", "has_a_relationship_with_infox", "lift"])\
    .sort_values(["has_a_relationship_with_infox", "lift"])\
    .style.format({"lift": "{:+.2%}"})

,column,has_a_relationship_with_infox,lift
4,écriture de qualité,False,+0.70%
0,signé,False,+1.63%
6,faits,False,+2.09%
20,degré infox,False,+nan%
13,des chiffres et des tableaux,True,+0.47%
14,sophisme,True,+0.47%
11,scientifique sulfureux,True,+1.40%
12,des pages et des pages,True,+5.35%
19,titre décalé,True,+5.58%
3,points de vue différents présentés,True,+6.28%


In [ ]:
for col in label_columns:
    df.loc[:,col]=df[col].apply(normalize)

In [ ]:
col = 'infox'
df.loc[:,col]=df[col].apply(normalize)

In [ ]:
stats.chi2_contingency(pd.crosstab(df['fausse nouvelle'], df["infox"]))

In [ ]:
pd.crosstab(df["invérifiable"], df["infox"], margins=True)

In [ ]:
label_columns = ['infox','signé',
       'lieux, dates et personnes ', 'lieux, dates et personnes cohérentes',
       'points de vue différents présentés', 'écriture de qualité',
        'faits', 'opinions', 'fausse nouvelle', 'insinuations',
       'exagération', 'scientifique sulfureux', 'des pages et des pages',
       'des chiffres et des tableaux', 'sophisme', 'inversion de la preuve',
       'invérifiable', 'propos rapporté', 'sources citées', 'titre décalé',]

In [ ]:
result = {}
for col1 in label_columns:
    result[col1] = {}
    for col2 in label_columns:
        chi2, p, dof, expected = stats.chi2_contingency(pd.crosstab(df[col1], df[col2]))
        #result[col1][col2] = {}        
        #result[col1][col2]['chi2'] = chi2
        result[col1][col2] = p
        #result[col1][col2]['dof'] = dof
        #result[col1][col2]['expected'] = expected

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
df_chi2 = pd.DataFrame(result)

In [ ]:
df_chi2.round(3).to_csv('data/analysis/220907-Chi2.csv',decimal=',')

In [ ]:
df_chi2.round(3)

In [ ]:
result = {}
for col1 in label_columns:
    result[col1] = {}
    for col2 in label_columns:
        chi2, p, dof, expected = stats.chi2_contingency(pd.crosstab(df[col1], df[col2]))
        #result[col1][col2] = {}        
        result[col1][col2]['chi2'] = chi2
        #result[col1][col2] = p
        #result[col1][col2]['dof'] = dof
        #result[col1][col2]['expected'] = expected